# A Basic Capacity Expansion Model
## Power Systems Optimization

This notebook was adapted from Jesse D. Jenkins and Michael R. Davidson to use Python instead of Julia. The original notebook can be found here: https://github.com/Power-Systems-Optimization-Course/power-systems-optimization/blob/de0fe0c27e94849730d150db7516061d8c8bc9be/Notebooks/03-Basic-Capacity-Expansion.ipynb

This notebook presents a basic electricity generation capacity expansion model: the task of minimizing the cost of fixed and variable costs across a fleet of generators to meet anticipated future electricity demand.

We first present a basic optimization formulation of this optimal thermal generator capacity expansion problem, assuming a "greenfield" setting with no existing generating capacity. We also show how to create this model with Python using gurobipy with an academic license.

Next, we discuss the impact of variable renewable energy sources (wind, solar) and present a slightly modified formulation that co-optimizes thermal and renewable capacities.

Finally, we will note how this problem generalizes to a "brownfield" expansion problem including existing generators, with optimal retirement decisions included in the problem.

Note these capacity expansion problems will be basic/simplified formulations, ignoring inter-temporal operating constraints (e.g. ramp limits), energy storage, network constraints and geospatial tradeoffs, unit commitment decisions for thermal generators, and operating reserves.

## 1. Optimal thermal generator expansion problem

Assume a utility can build any of the following four thermal generation types: geothermal, coal, natural gas combined cycle gas turbine (CCGT), and natural gas combustion turbine (CT). The utility must decide on the mix of thermal generators to meet forecasted demand in a future planning year across each hour of the year.

Assume for simplicity that there are no existing generators -- that is, this is a "greenfield" expansion problem where all generators are newly built.

Finally, assume that any involuntary curtailment of demand caused by shortfalls in supply is penalized at the cost of non-served energy (NSE). (Note that by penalizing the opportunity cost of demand curtailment, this cost minimization problem is equivalent to a social welfare maximization problem, assuming inelastic demand).

## Problem formulation

This yields the following **optimization problem**:

\begin{align}  
\min &\sum_{g \in G} \left(FixedCost_g \times CAP_g + \sum_{h \in H} VarCost_g \times GEN_{g,h}\right)&\\
&\quad\quad+ \sum_{h \in H} NSECost \times NSE_h& \tag{1}\\
&\text{s.t.}&\\
&\sum_{g \in G} GEN_{g,h} + NSE_h = Demand_h \quad \forall h \in H \tag{2}\\
&GEN_{g,h} \leq CAP_g \quad \forall g \in G, \quad \forall h \in H \tag{3}\\
&CAP_{g} \geq 0 \quad \forall g \in G \tag{4}\\
&NSE_{h} \geq 0 \quad \forall h \in H \tag{5}\\
&GEN_{g,h} \geq 0 \quad \forall g \in G, \quad \forall h \in H \tag{6}\\
\end{align}

where

\begin{align}
&FixedCost_g = Capex_g \times CRF_g + FixedOM_g \quad \forall g \in G \tag{7}\\
&CRF_g = \frac{WACC_g(1+WACC_g)^{Life_g}}{(1+WACC_g)^{Life_g}-1} \quad \forall g \in G \tag{8}\\
&VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g \quad \forall g \in G \tag{9}
\end{align}
&nbsp;  

The **decision variables** in the above problem are:  
* $CAP_{g}$ is the capacity (in MW) built for each generation type, $g$'    
* $GEN_{g,h}$ is the generation (in MWh) produced by each generator, $g$, in each hour, $h$; and  
* $NSE_h$ is the quantity of involuntarily curtailed demand in each hour (in MWh), $h$.  
&nbsp;  
Note that if we ignore the discrete nature of large thermal power plant decisions (investment and power plant cycling decisions), we have continous decisions and *a linear programming (LP) problem* with linear objective function and linear constraints. (We'll incorporate discrete decisions in the notebook on Unit Commitment)
&nbsp;  

The **sets** are:
* $G$ is the set of generators: \[geo, coal, CCGT, CT\]; and  
* $H$ is the set of hours in the year: \[1:8760\].  
&nbsp;  

The **parameters** are:  
* $Capex_g$ is the capital expenditure to construct the power plant (overnight cost + cost of financing during construction) (\$/MW);  
* $CRF_g$ is the annual capital recovery factor or share of $Capex$ recovered in each year to pay off equity and debt (per unit);  
* $WACC_g$ is the weighted average cost of capital to finance the generator, or the weighted cost of interest on debt and returns to equity investors (\%);  
* $Life_g$ is the financial asset life of the asset (years);  
* $FixedOM_g$ is the fixed operations and maintenance cost (\$/MW-year);  
* $VarOM_g$ is the variable operations and maintenance cost (\\$/MWh);  
* $HeatRate_g$ is the rate of fuel consumed per unit of electricity produced (MMBtu/MWh);  
* $FuelCost_g$ is the cost of fuel (\\$/MMBtu);  
* $NSECost$ is the opportunity cost or penalty incurred for involuntary; non-served energy (\$/MWh); and  
* $Demand_h$ is the demand in each hour.
&nbsp;  

## Building the problem with Python  
Now let's define this LP problem with the help of Python + gurobipy.  

**1. Load packages**  

First, let's install and load the Python packages we'll need to work with here.  

In [ ]:
!pip install gurobipy
!pip install gurobipy_pandas

Now you'll need to get a Gurobi Academic license. You can request one here: https://portal.gurobi.com/iam/register/

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gppd # allows us to use pandas data frames with gurobipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

**2. Load parameters (data)**

Then we'll load our parameters, starting with the cost parameters for the candidate generators.

The following generator data are illustrative and approximate, but most are based on the National Renewable Energy Laboratory's [Annual Technology Baseline](https://atb.nrel.gov/electricity/2020/) 2020, except for natural gas CT capex, which is based on Lazard's [Levelized Cost of Electricity Version 13.0](https://www.lazard.com/media/o3ln2wve/lazards-levelized-cost-of-energy-version-130-vf.pdf).

In [ ]:
# allow access to google drive
drive.mount('/content/drive')

generators = pd.read_csv("drive/MyDrive/Colab Notebooks/CE4110_6250/expansion_data/generators_for_expansion.csv", index_col = "G")
generators.index.name = "Source"
generators

Note we've got wind and solar costs in here, but we'll get to these variable renewable resources in the next section.

Next we'll load the hourly demand data. This demand time series is the actual reported demand (from 2012) for San Diego Gas and Electric (via the PowerGenome data platform as reported to FERC Form 714).

In [ ]:
demand = pd.read_csv("drive/MyDrive/Colab Notebooks/CE4110_6250/expansion_data/demand_for_expansion.csv", index_col = "Hour")
demand = demand["Demand"] # coerce from datatype pd.DataFrame to pd.Series (important later on)
demand.head()

Let's plot the demand to get a feel for what we're working with here...

In [ ]:
plt.plot(demand, label="")
plt.ylim([0,5000])
plt.title("Hourly Demand")
plt.xlabel("Hour")
plt.ylabel("MWh")

(Note that this time series starts at 0:00-0:59 UTC time, or 5pm Pacific time.)

You can see some of the seasonal trends here in San Diego electricity demand with some large peaks in demand in the summer months driven by air conditioning needs. This profile also includes the impact of about 600 MW of distributed solar PV installed behind the meter, which reduces the net load for the bulk generators we are planning here.

The chronological demand profile is a little hard to look at though, with all the daily variation, and since we're not considering intertemporal constraints (yet), let's re-sort the demand from highest to lowest hour of demand across the year, what is known as a **"load duration curve."**

In [ ]:
demand

In [ ]:
p = demand.index / (demand.shape[0] + 1) # probability of exceedance
plt.plot(p,demand.sort_values(ascending=False), label="")
plt.ylim([0,5000])
plt.title("Load duration curve (LDC)")
plt.xlabel("Probability of Exceedance")
plt.ylabel("MWh")

In [ ]:
plt.plot(demand.sort_values(), p, label="")
plt.title("Load duration as cumulative distribution")
plt.ylabel("Cumulative probability")
plt.xlabel("MWh")

Examining the load duration curve, we can more easily see the wide range of variation in hourly demand across the year...

In [ ]:
print("Minimum demand: %d" % demand.min())
print("Maximum demand: %d" % demand.max())
print("Median demand: %d" % demand.median())
print("Mean demand: %d" % demand.mean())

Finally, we will set the penalty for non-served energy, or $NSECost$
 to $9,000/MWh, and pull out the data we need as parameters for our optimization model: demands, fixed costs, and variables costs.

In [ ]:
generators

In [ ]:
demand.head()

In [ ]:
NSEcost = 9000 # penalty for no service

**3. Set up an environment to use your Gurobi Academic license**

$\color{red} {\text{CHANGE THE LICENSE VALUES TO THOSE IN THE LICENSE YOU ACQUIRED!}}$

In [ ]:
# create environment with Gurobi academic license
# CHANGE THESE TO THE VALUES FOR YOUR LICENSE
params = {
"WLSACCESSID": '69c0357e-257d-4855-a3f0-446b8e822abd',
"WLSSECRET": '11082e14-04a8-4439-a62d-f3acd3d46920',
"LICENSEID": 2471772,
}
env = gp.Env(params=params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2471772
Academic license 2471772 - for non-commercial use only - registered to jd___@virginia.edu


In [ ]:
# set interactive mode (causes performance hit but allows you to view the model as you build it)
gppd.set_interactive()

# Create the model within the Gurobi environment
model = gp.Model('Capacity Expansion',env=env)

**4. Define decision variables**

Next, define decision variables and their bound constraints:

$CAP_g \geq 0$, the capacity (in MW) built for each generation type, $g$, and    
$GEN_{g,h} \geq 0$, the generation (in MWh) produced by each generator, $g$, in each hour, $h$; and  
$NSE_h \geq 0$, the quantity of involuntarily curtailed demand in each hour, $h$.  

In [ ]:
# create dataframes to contain gurobi variables, gurobi constrants, and parameters
## by source
df_by_src = generators.loc[:, ['FixedCost', 'VarCost']].copy()
df_by_src.loc["NSE", 'FixedCost'] = 0 # consider NSE a generator with no fixed cost
df_by_src.loc["NSE", 'VarCost'] = NSEcost # variable cost of NSE is NSEcost
df_by_src.drop(["Wind","Solar"],axis=0,inplace=True) # drop wind and solar for now
df_by_src

In [ ]:
## by source and hour
### create multi-indexed dataframe for each decision variable index
list_of_indices = [
    list(df_by_src.index),
    list(demand.index) # hours
]

multi_idx = pd.MultiIndex.from_product(list_of_indices, names=["Source", "Hour"])
multi_idx

In [ ]:
df_by_hr = pd.DataFrame(index = multi_idx)
df_by_hr

In [ ]:
# default bounds are [0,infinity) unless otherwise specified, so do not need to specify bounds
GEN = df_by_hr.gppd.add_vars(model, name = "gen_MWh", vtype=GRB.CONTINUOUS)
GEN

In [ ]:
# default bounds are [0,infinity) unless otherwise specified, so do not need to specify bounds
CAP = df_by_src.gppd.add_vars(model, name = "cap_MW", vtype=GRB.CONTINUOUS)
CAP

In [ ]:
GEN.join(CAP['cap_MW'])

**5. Define constraints**

Generation over all generators + Curtailed Demand = Total Demand every hour:  
$\sum_{g \in G} GEN_{g,h} + NSE_h = Demand_h \quad \forall h \in H \tag{10}$  
&nbsp;  
Generation at generator $g$ cannot exceed its capacity every hour:  
$GEN_{g,h} \leq CAP_g \quad \forall g \in G, \quad \forall h \in H \tag{11}$


In [ ]:
# non-negativity constraints assumed, just add demand and capacity constraints
# pandas.Dataframe.join documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html
c1_capacity = gppd.add_constrs(
    model, GEN['gen_MWh'], GRB.LESS_EQUAL, GEN.join(CAP['cap_MW'])['cap_MW'], name="c1_capacity"
)
c1_capacity

In [ ]:
# pandas.Dataframe.groupby documentation:  https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
c2_demand = gppd.add_constrs(
    model, GEN['gen_MWh'].groupby(level = "Hour").sum(), # this sums all power generated from all sources each hour
    GRB.EQUAL, demand, name="c2_demand" # demand needed to be a pd.Series datatype here
)
c2_demand

**6. Define objective function**

Minimize total costs of generation (which consists of fixed capital costs and variable generation costs), and curtailed demand.  
&nbsp;  
$\min \sum_{g \in G} \left(FixedCost_g \times CAP_g + \sum_{h \in H} VarCost_g \times GEN_{g,h}\right)\\
\quad\quad+ \sum_{h \in H} NSECost \times NSE_h \tag{12}$

In [ ]:
# objective function
gen_costs = (df_by_src["VarCost"] * GEN["gen_MWh"]).agg(gp.quicksum) # includes NSE penalty
fixed_costs = (df_by_src["FixedCost"] * CAP['cap_MW']).agg(gp.quicksum)

model.setObjective(gen_costs + fixed_costs)

**7. Finally, define and solve the optimization problem**

In [ ]:
model.write('capacity_expansion.lp')
model.optimize()

Find generation at each generator in each hour.

In [ ]:
# process capacity expansion results
cap = vars_by_src.cap_MW.gppd.x
cap_share = cap/np.max(demand)*100 # percent of max demand (we want to be sure we can meet this)
cap_share.name = "cap_share_of_peak_demand" # name the pandas series (this will become the column header when we
                                            # add it to our results dataframe)

# process energy generation results
gen = vars_by_hr.gen_MWh.gppd.x.groupby(level = "Source").sum() # sum power generated by source
genshare = gen / np.sum(demand)*100 # compute the fraction of demand met by each source
genshare.name = "gen_share_of_total_demand"
gen = gen / 1000 # convert from MWh to GWh
gen.name = "Gen_GWh"

# Pandas concat documentation: https://pandas.pydata.org/docs/reference/api/pandas.concat.html
df_results = pd.concat([cap,cap_share, gen, genshare], axis = 1) # axis = 1 because we are combining columns rather than rows

df_results.loc['SUM', :] = df_results.sum() # add a row equal to the sum of each column as a check

df_results

You can see that given the cost assumptions provided, the least-cost capacity mix includes exclusively gas-fired generators: a mix of CCGTs which operate at higher utilization rates given their comparatively higher fixed costs and lower variable costs and CTs which run infrequently (to meet peaks in demand) given their lower fixed costs and higher fuel costs.

CCGTs make up 69% of capacity and CTs 27%, while CCGTs produce over 98% of annual generation and CTs provide only about 2%. A maximum of 4% of demand is curtailed in any hour (195 MW) and less than 0.003% of demand across the year is unserved (99.9972% reliability or "four nines" reliability, which is very good).

What might happen if you changed the fuel costs? Or reduced the penalty for non-served energy? Questions to consider for Homework 3.

## 2. Co-optimizing thermal generators and variable renewables

Now let's add some wind and solar to the mix.

We already loaded the cost data for wind and solar previously.

In [ ]:
generators

In [ ]:
# re-create df_by_src without dropping wind and solar
df_by_src = generators.loc[:, ['FixedCost', 'VarCost']].copy()
df_by_src.loc["NSE", 'FixedCost'] = 0 # consider NSE a generator with no fixed cost
df_by_src.loc["NSE", 'VarCost'] = NSEcost # variable cost of NSE is NSEcost
df_by_src

We will also need time series data containing the hourly capacity factor or maximum availability of wind and solar per unit of capacity deployed. This reflects the weather-dependent variation in potential wind and solar output across each hour of the year.

In [ ]:
capacity_factors = pd.read_csv("drive/MyDrive/Colab Notebooks/CE4110_6250/expansion_data/wind_solar_for_expansion.csv", index_col = "Hour")
capacity_factors.head()

Let's plot the first month of data (January), to get a feel for the patterns of variability.

In [ ]:
plt.plot(capacity_factors["Wind"].iloc[0:(24*31)], label="Wind")
plt.ylim([0,1])
plt.plot(capacity_factors["Solar"][0:(24*31)], label="Solar")
plt.title("Capacity Factors for Wind & Solar - January")
plt.xlabel("Hour")
plt.ylabel("Capacity Factor")
plt.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

And here's what July (the peak month for demand) looks like...

In [ ]:
july1 = 24*(31*3+30*2+28)-1
july31 = 24*(31*4+30*2+28)
plt.plot(capacity_factors["Wind"].iloc[july1:july31], label="Wind")
plt.ylim([0,1])
plt.plot(capacity_factors["Solar"][july1:july31], label="Solar")
plt.title("Capacity Factors for Wind & Solar - January")
plt.xlabel("Hour")
plt.ylabel("Capacity Factor")
plt.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

Now we have to slightly modify our problem formulation. For variable renewable resources $g \in RE$, where $RE$ is the subset $\{Wind,Solar\}\subset G$, the maximum possible generation is now time-dependent and equal to $CAP_g \times cf_{g,h}$ where $cf_{g,h}$ is the capacity factor for resource $g$ in hour $h$. That is, a 100 MW solar farm with an hourly capacity factor of 0.2 (20%) can produce no more than 20 MW in that hour. (We assume these are curtailable, utility-scale wind and solar PV plants that can produce less than this maximum amount if desired).

This yields (red text is only change from previous formulation):  

\begin{align}  
\min \sum_{g \in G} \left(FixedCost_g \times CAP_g + \sum_{h \in H} VarCost_g \times GEN_{g,h}\right)\\
\quad\quad+ \sum_{h \in H} NSECost \times NSE_h \tag{13}\\
\text{s.t.}\\
\sum_{g \in G} GEN_{g,h} + NSE_h = Demand_h \quad \forall h \in H \tag{14}\\
\color{red}{GEN_{g,h} \leq CAP_g \quad \forall g \notin RE, \quad \forall h \in H} \tag{15}\\
\color{red}{GEN_{g,h} \leq CAP_g \times cf_{g,h} \quad \forall g \in RE, \quad \forall h \in H} \tag{16}\\
CAP_{g} \geq 0 \quad \forall g \in G \tag{17}\\
NSE_{h} \geq 0 \quad \forall h \in H \tag{18}\\
GEN_{g,h} \geq 0 \quad \forall g \in G, \quad \forall h \in H \tag{19}\\
\end{align}

Here are some helpful coding tips to do this. First, add the capacity factors to df_by_hr. Set them to 1 for non-renewable generators.

In [ ]:
### re-organize wide to long using pd.DataFrame.stack()
# documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html
capacity_factors = capacity_factors.T.stack()
capacity_factors.index.names = ["Source", "Hour"]
capacity_factors.name = "capfac"
capacity_factors

In [ ]:
# redefine df_by_hr so that it includes Wind and Solar
## by source and hour
### create multi-indexed dataframe for each decision variable index
list_of_indices = [
    list(generators.index),
    list(demand.index) # hours
]

multi_idx = pd.MultiIndex.from_product(list_of_indices, names=["Source", "Hour"])
# create dataframe
df_by_hr = pd.DataFrame(index = multi_idx)
# add the capacity factors
df_by_hr = df_by_hr.join(capacity_factors)
df_by_hr # Gurobi won't work if there are missing values! What should this be for the non-renewables?

In [ ]:
# fillna(1) puts a capacity factor of 1 for generators that have no capacity factor data (generators other than wind and solar)
df_by_hr = df_by_hr.fillna(1)
df_by_hr

In Homework 3, you can code up this version of the generation expansion problem.

## 3. Brownfield expansion and retirement decisions

The formulation above assumes no existing generating capacity -- a "greenfield" expansion problem.

But realistic planning problems will begin with today's installed capacity and then consider both what to build and what to retire to meet future demand at the lowest cost.

The capital expenditure to build existing generators is a sunk cost, so this should not factor into the economic decision making as to whether to continue operating a power plant. The relevant decisions have to do only with avoidable costs, in this case, ongoing fixed operations and maintenance costs (which we will consider inclusive of any capital expenditure to refurbish or replace aging equipment and continue operating).

Let's define a formulation for this "brownfield" generation expansion problem with existing capacity retirement decisions and new build capacity expansion decisions.

First, define a new subset of generators $OLD \subset G$ that consists of all existing generators, and an existing capacity parameter $ExistingCap_g \forall g \in OLD$.

For these resources, we don't have a capacity expansion decision ($CAP_g$), but we do have a retirement decision, $RET_g$, which indicates how much capacity is to be retired before the planning year.

The $FixedCost_g$ for each existing generator consists only of the ongoing fixed operations and maintenance costs ($FixedOM_g$), with no annuitized investment cost component.

For convenience, let's also define a new subset $NEW \subset G$ that consists of all new build capacity options.

The new formulation is as follows:

\begin{align}
\min &\sum_{g \in NEW} FixedCost_g \times CAP_g + \sum_{g \in OLD} FixedCost_g \times (ExistingCap_g - RET_g)& \\
&\quad\quad+ \sum_{g\in G}\sum_{h \in H} VarCost_g \times GEN_{g,h} + \sum_{h \in H} NSECost \times NSE_h & \tag{20}\\
\end{align}

\begin{align}
&\text{s.t.}&\\
&\text{Demand balance constraint:}&\\
& \sum_{g \in G} GEN_{g,h} + NSE_h = Demand_h & \forall \quad h \in H \tag{21}\\
&\text{Capacity constraint for new build thermal:}&\\
& GEN_{g,h} \leq CAP_g & \forall \quad g \in NEW \text{ & } \notin RE, \quad \forall h \in H \tag{22}\\
&\text{Capacity constraint for existing thermal:}&\\
& GEN_{g,h} \leq ExistingCap_g - RET_g & \forall \quad g \in OLD \text{ & } \notin RE, \quad \forall h \in H \tag{23}\\
&\text{Capacity constraint for new build RE:}&\\
& GEN_{g,h} \leq CAP_g \times cf_g & \forall \quad g \in NEW \text{ & } \in RE, \quad \forall h \in H \tag{24}\\
&\text{Capacity constraint for existing RE:}&\\,
& GEN_{g,h} \leq (ExistingCap_g - RET_g) \times cf_g & \forall \quad g \in OLD \text{ & } \in RE, \quad \forall h \in H \tag{25}\\
&\text{Capacity variable only for new build:}&\\
& CAP_{g} \geq 0 & \forall \quad g \in NEW \tag{26}\\
&\text{Retirement variable only for existing:}&\\
& RET_{g} \geq 0 & \forall \quad g \in OLD \tag{27}\\
&\text{Generation and NSE variables:}&\\
& GEN_{g,h} \geq 0 & \forall \quad g \in G, \quad \forall h \in H \tag{28}\\
& NSE_{h} \geq 0 & \forall h \in H \tag{29}
\end{align}
&nbsp;  
You'll construct this model in Homework 3 as well. The existing gas capacity data and fixed and variable costs of existing gas capacity are given below. Note: there is no existing renewable capacity here to consider, only thermal.

In [ ]:
# Add data on old generators to dataframe
generators.loc["Old_CCGT",:] = ["Existing CCGT",0,40000,5,7.5,4,0,0,0,0,40000,30]
generators.loc["Old_CT",:] = ["Existing CT",0,30000,11,11.0,4,0,0,0,0,30000,55]
generators

In [ ]:
# Set installed capacity for existing CCGTs:
ExistingCap_CCGT = 1260 # Approximate actual existing capacity in SDGE
ExistingCap_CT = 925 # Approximate actual existing capacity in SDGE
# Add new column to generators Data Frame
generators["ExistingCap"] = [0,0,0,0,0,0,ExistingCap_CCGT,ExistingCap_CT]
generators